# 라이브러리 로드

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import gc
from tensorflow.keras.backend import clear_session

from matplotlib import rcParams, pyplot as plt

import numpy as np
import pandas as pd
import random as rn
import re

from pathlib import Path
import warnings

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 

In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

# 학습데이터 로드

In [4]:
seed = 2020
# np.random.seed(seed)
# rn.seed(seed)
# tf.random.set_seed(seed)
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
#                              inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)

In [5]:
data_dir = Path('../data/dacon-novel-author-classification')
feature_dir = Path('../build/feature')
val_dir = Path('../build/val')
tst_dir = Path('../build/tst')
sub_dir = Path('../build/sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test_x.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'author'
n_fold = 5
n_class = 5

In [6]:
algorithm_name = 'mlp'
feature_name = 'feature'
model_name = f'{algorithm_name}_{feature_name}'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'
feature_Ver4_file = feature_dir / f'{feature_name}_Ver4.csv'


mlp_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
mlp_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
mlp_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'
mlp_oof_pred_ver4_file = val_dir / f'{model_name}_oof_pred_ver4.csv'


mlp_test_pred_ver1_file = tst_dir / f'{model_name}_test_pred_ver1.csv'
mlp_test_pred_ver2_file = tst_dir / f'{model_name}_test_pred_ver2.csv'
mlp_test_pred_ver3_file = tst_dir / f'{model_name}_test_pred_ver3.csv'
mlp_test_pred_ver4_file = tst_dir / f'{model_name}_test_pred_ver4.csv'


mlp_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
mlp_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
mlp_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'
mlp_submission_ver4_file = sub_dir / f'{model_name}_submission_Ver4.csv'

## Ver 1 데이터 로드

In [7]:
dataset = pd.read_csv(feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he was almost choking there was so much so muc...,3.0
1,your sister asked for it i suppose,2.0
2,she was engaged one day as she walked in peru...,1.0
3,the captain was in the porch keeping himself c...,4.0
4,have mercy gentlemen odin flung up his hands d...,3.0


In [8]:
# train set
Ver1_X = dataset.loc[dataset[target_col] != -1 , :]
Ver1_X.drop(columns=target_col,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_col] == -1, :]
Ver1_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver 2 데이터 로드

In [9]:
dataset = pd.read_csv(feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged one day walked perusing janes last let...,1.0
3,captain porch keeping carefully way treacherou...,4.0
4,mercy gentlemen odin flung hands dont write an...,3.0


In [10]:
# train set
Ver2_X = dataset.loc[dataset[target_col] != -1 , :]
Ver2_X.drop(columns=target_col,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_col] == -1, :]
Ver2_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver3 데이터 로드

In [11]:
dataset = pd.read_csv(feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,almost choking much much wanted say strange ex...,3.0
1,sister asked suppose,2.0
2,engaged day walked perusing janes last letter ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,mercy gentlemen flung up hands dont write anyw...,3.0


In [12]:
# train set
Ver3_X = dataset.loc[dataset[target_col] != -1 , :]
Ver3_X.drop(columns=target_col,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_col] == -1, :]
Ver3_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

(54879, 1) (54879,) (19617, 1)


## Ver4 데이터 로드

In [13]:
dataset = pd.read_csv(feature_Ver4_file, index_col=0)
print(dataset.shape)
dataset.head()

(74496, 2)


,text,author
index,,
0,he almost choking there so much so much he wan...,3.0
1,sister asked for it suppose,2.0
2,she engaged one day she walked perusing janes ...,1.0
3,captain porch keeping himself carefully out wa...,4.0
4,have mercy gentlemen odin flung up hands dont ...,3.0


In [14]:
# train set
Ver4_X = dataset.loc[dataset[target_col] != -1 , :]
Ver4_X.drop(columns=target_col,inplace=True,axis=1)
Ver4_y = dataset.loc[dataset[target_col] != -1, target_col]
Ver4_y.astype(int)

# test set
Ver4_test = dataset.loc[dataset[target_col] == -1, :]
Ver4_test.drop(columns=target_col, inplace=True,axis=1)

print(Ver4_X.shape, Ver4_y.shape, Ver4_test.shape)

(54879, 1) (54879,) (19617, 1)


# 모델 학습

In [15]:
#파라미터 설정
vocab_size = 30000
embedding_dim = 128
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [16]:
def get_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
    ])
    
    # compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=.0003))
    return model

In [17]:
datasets = [ (Ver1_X, Ver1_test, Ver1_y), (Ver2_X, Ver2_test, Ver2_y),
            (Ver3_X, Ver3_test, Ver3_y), (Ver4_X, Ver4_test, Ver3_y)]

mlp_oof_preds = []
mlp_test_preds = []

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for number ,(X, test, y) in enumerate(datasets,1):
    print(f'start : {number}')
    # 토큰화
    X_train = np.array([x for x in X['text']])
    X_test = np.array([x for x in test['text']])
    y = np.array(y.values)
    
    tokenizer = Tokenizer(num_words = vocab_size)
    tokenizer.fit_on_texts(X_train)
    word_index = tokenizer.word_index
    
    # 시퀸스화 + 패딩
    train_sequences = tokenizer.texts_to_sequences(X_train)
    test_sequences = tokenizer.texts_to_sequences(X_test)
    trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
    tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
    print(trn.shape, tst.shape)
    
    # oof , test 저장
    mlp_oof_pred = np.zeros((trn.shape[0], n_class))
    mlp_test_pred = np.zeros((tst.shape[0], n_class))
    
    for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
        print(f'traing model for CV #{i}')
        clf = get_model()
        
        es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)
        
        clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=200,
            batch_size=1024,
            callbacks=[es])
        
        mlp_oof_pred[i_val, :] = clf.predict(trn[i_val])
        mlp_test_pred += clf.predict(tst) / n_fold
        
        del clf
        clear_session()
        gc.collect()
    
    mlp_oof_preds.append(mlp_oof_pred)
    mlp_test_preds.append(mlp_test_pred)
        
    print(f'end : {number}')

start : 1
(54879, 500) (19617, 500)
traing model for CV #1
Epoch 1/200
43/43 [==============================] - 5s 122ms/step - loss: 1.5828 - val_loss: 1.5682
Epoch 2/200
43/43 [==============================] - 5s 115ms/step - loss: 1.5682 - val_loss: 1.5671
Epoch 3/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5665 - val_loss: 1.5652
Epoch 4/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5644 - val_loss: 1.5617
Epoch 5/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5585 - val_loss: 1.5529
Epoch 6/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5439 - val_loss: 1.5304
Epoch 7/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5050 - val_loss: 1.4714
Epoch 8/200
43/43 [==============================] - 5s 115ms/step - loss: 1.4290 - val_loss: 1.3842
Epoch 9/200
43/43 [==============================] - 5s 115ms/step - loss: 1.3373 - val_loss: 1.2935
Epoch 10/200
43/43 [============

43/43 [==============================] - 5s 113ms/step - loss: 0.6514 - val_loss: 0.8110
Epoch 30/200
43/43 [==============================] - 5s 114ms/step - loss: 0.6270 - val_loss: 0.7981
Epoch 31/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6082 - val_loss: 0.7903
Epoch 32/200
43/43 [==============================] - 5s 113ms/step - loss: 0.5889 - val_loss: 0.7910
Epoch 33/200
43/43 [==============================] - 5s 113ms/step - loss: 0.5702 - val_loss: 0.7749
Epoch 34/200
43/43 [==============================] - 5s 114ms/step - loss: 0.5530 - val_loss: 0.7649
Epoch 35/200
43/43 [==============================] - 5s 113ms/step - loss: 0.5353 - val_loss: 0.7597
Epoch 36/200
43/43 [==============================] - 5s 113ms/step - loss: 0.5207 - val_loss: 0.7670
Epoch 37/200
43/43 [==============================] - 5s 114ms/step - loss: 0.5056 - val_loss: 0.7523
Epoch 38/200
43/43 [==============================] - 5s 114ms/step - loss: 0.4939 - val_loss: 

Epoch 17/200
43/43 [==============================] - 5s 113ms/step - loss: 1.0962 - val_loss: 1.1141
Epoch 18/200
43/43 [==============================] - 5s 114ms/step - loss: 1.0646 - val_loss: 1.0908
Epoch 19/200
43/43 [==============================] - 5s 113ms/step - loss: 1.0322 - val_loss: 1.0705
Epoch 20/200
43/43 [==============================] - 5s 114ms/step - loss: 1.0053 - val_loss: 1.0528
Epoch 21/200
43/43 [==============================] - 5s 113ms/step - loss: 0.9842 - val_loss: 1.0387
Epoch 22/200
43/43 [==============================] - 5s 113ms/step - loss: 0.9611 - val_loss: 1.0362
Epoch 23/200
43/43 [==============================] - 5s 114ms/step - loss: 0.9396 - val_loss: 1.0163
Epoch 24/200
43/43 [==============================] - 5s 112ms/step - loss: 0.9169 - val_loss: 1.0112
Epoch 25/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8963 - val_loss: 0.9979
Epoch 26/200
43/43 [==============================] - 5s 114ms/step - loss: 0.8742

43/43 [==============================] - 5s 114ms/step - loss: 0.7075 - val_loss: 0.9087
Epoch 33/200
43/43 [==============================] - 5s 112ms/step - loss: 0.6885 - val_loss: 0.9057
Epoch 34/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6736 - val_loss: 0.9101
Epoch 35/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6593 - val_loss: 0.8999
Epoch 36/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6449 - val_loss: 0.8944
Epoch 37/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6341 - val_loss: 0.9047
Epoch 38/200
43/43 [==============================] - 5s 113ms/step - loss: 0.6226 - val_loss: 0.8921
Epoch 39/200
43/43 [==============================] - 5s 114ms/step - loss: 0.6054 - val_loss: 0.8925
Epoch 40/200
43/43 [==============================] - 5s 112ms/step - loss: 0.5979 - val_loss: 0.8940
Epoch 41/200
43/43 [==============================] - 5s 114ms/step - loss: 0.5826 - val_loss: 

Epoch 22/200
43/43 [==============================] - 5s 113ms/step - loss: 0.9978 - val_loss: 1.0579
Epoch 23/200
43/43 [==============================] - 5s 112ms/step - loss: 0.9708 - val_loss: 1.0348
Epoch 24/200
43/43 [==============================] - 5s 113ms/step - loss: 0.9428 - val_loss: 1.0171
Epoch 25/200
43/43 [==============================] - 5s 112ms/step - loss: 0.9197 - val_loss: 1.0070
Epoch 26/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8950 - val_loss: 0.9870
Epoch 27/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8692 - val_loss: 0.9674
Epoch 28/200
43/43 [==============================] - 5s 112ms/step - loss: 0.8383 - val_loss: 0.9527
Epoch 29/200
43/43 [==============================] - 5s 114ms/step - loss: 0.8131 - val_loss: 0.9340
Epoch 30/200
43/43 [==============================] - 5s 114ms/step - loss: 0.7821 - val_loss: 0.9117
Epoch 31/200
43/43 [==============================] - 5s 114ms/step - loss: 0.7550

43/43 [==============================] - 5s 112ms/step - loss: 1.5691 - val_loss: 1.5687
Epoch 4/200
43/43 [==============================] - 5s 114ms/step - loss: 1.5684 - val_loss: 1.5680
Epoch 5/200
43/43 [==============================] - 5s 113ms/step - loss: 1.5677 - val_loss: 1.5668
Epoch 6/200
43/43 [==============================] - 5s 112ms/step - loss: 1.5659 - val_loss: 1.5649
Epoch 7/200
43/43 [==============================] - 5s 112ms/step - loss: 1.5628 - val_loss: 1.5605
Epoch 8/200
43/43 [==============================] - 5s 113ms/step - loss: 1.5544 - val_loss: 1.5474
Epoch 9/200
43/43 [==============================] - 5s 113ms/step - loss: 1.5332 - val_loss: 1.5166
Epoch 10/200
43/43 [==============================] - 5s 113ms/step - loss: 1.4862 - val_loss: 1.4572
Epoch 11/200
43/43 [==============================] - 5s 112ms/step - loss: 1.4169 - val_loss: 1.3919
Epoch 12/200
43/43 [==============================] - 5s 113ms/step - loss: 1.3545 - val_loss: 1.3462

43/43 [==============================] - 5s 114ms/step - loss: 0.9407 - val_loss: 0.9969
Epoch 20/200
43/43 [==============================] - 5s 114ms/step - loss: 0.8992 - val_loss: 0.9813
Epoch 21/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8678 - val_loss: 0.9558
Epoch 22/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8381 - val_loss: 0.9393
Epoch 23/200
43/43 [==============================] - 5s 113ms/step - loss: 0.8099 - val_loss: 0.9269
Epoch 24/200
43/43 [==============================] - 5s 114ms/step - loss: 0.7830 - val_loss: 0.9087
Epoch 25/200
43/43 [==============================] - 5s 113ms/step - loss: 0.7576 - val_loss: 0.8948
Epoch 26/200
43/43 [==============================] - 5s 114ms/step - loss: 0.7366 - val_loss: 0.8857
Epoch 27/200
43/43 [==============================] - 5s 115ms/step - loss: 0.7092 - val_loss: 0.8732
Epoch 28/200
43/43 [==============================] - 5s 114ms/step - loss: 0.6853 - val_loss: 

In [18]:
for i,j in enumerate(mlp_oof_preds,1):
    print(f'ver{i} logloss = {log_loss(pd.get_dummies(y),j):8.4f}')
    print(f'ver{i} accuracy = {accuracy_score(y, np.argmax(j,axis=1))*100:8.4f}')

ver1 logloss =   0.7848
ver1 logloss =  71.0909
ver2 logloss =   0.8537
ver2 logloss =  68.8661
ver3 logloss =   0.8700
ver3 logloss =  67.3828
ver4 logloss =   0.8009
ver4 logloss =  70.7885


# 제출 파일 및 기타 파일 생성

In [19]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[sub.columns] = mlp_test_preds[0]
sub.to_csv(mlp_submission_ver1_file)

# Ver2
sub[sub.columns] = mlp_test_preds[1]
sub.to_csv(mlp_submission_ver2_file)

# Ver3
sub[sub.columns] = mlp_test_preds[2]
sub.to_csv(mlp_submission_ver3_file)
           
# Ver4
sub[sub.columns] = mlp_test_preds[3]
sub.to_csv(mlp_submission_ver4_file)

In [20]:
# mlp_oof_pred 파일 생성

# Ver1
np.savetxt(mlp_oof_pred_ver1_file, mlp_oof_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(mlp_oof_pred_ver2_file, mlp_oof_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(mlp_oof_pred_ver3_file, mlp_oof_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(mlp_oof_pred_ver4_file, mlp_oof_preds[3],fmt='%.18f', delimiter=',')

In [21]:
# mlp_test_pred 파일 생성

# Ver1
np.savetxt(mlp_test_pred_ver1_file, mlp_test_preds[0],fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(mlp_test_pred_ver2_file, mlp_test_preds[1],fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(mlp_test_pred_ver3_file, mlp_test_preds[2],fmt='%.18f', delimiter=',')

# Ver4
np.savetxt(mlp_test_pred_ver4_file, mlp_test_preds[3],fmt='%.18f', delimiter=',')